# Traffic Sign Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you鈥檙e going to use Keras to build a convolutional neural network in a few dozen lines of code.

You鈥檒l be connecting the concepts from the previous lessons to the methods that Keras provides.

## Dataset

The network you'll build with Keras is similar to the example that you can find in Keras鈥檚 GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 

However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.

You can download pickle files with sanitized traffic sign data here.

## Overview

Here are the steps you'll take to build the network:

1. First load the training data and do a train/validation split.
2. Preprocess data.
3. Build a feedforward neural network to classify traffic signs.
4. Build a convolutional neural network to classify traffic signs.
5. Evaluate performance of final neural network on testing data.

Keep an eye on the network鈥檚 accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you鈥檝e built and trained an effective model.

In [108]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
import math

## Load the Data

Start by importing the data from the pickle file.

In [109]:
# TODO: Implement load the data here.
with open('train.p', 'rb') as f:
    train = pickle.load(f)

with open('test.p', 'rb') as f:
    test = pickle.load(f)

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

## Validate the Network
Split the training data into a training and validation set.

Measure the [validation accuracy](https://keras.io/models/sequential/) of the network after two training epochs.

Hint: [Use the `train_test_split()` method](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from scikit-learn.

In [110]:
# TODO: Use `train_test_split` here.
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [111]:
# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."
assert(X_val.shape[0] == y_val.shape[0]), "The number of images is not equal to the number of labels."
assert(X_val.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

## Preprocess the Data

Now that you've loaded the training data, preprocess the data such that it's in the range between -0.5 and 0.5.

In [112]:
# TODO: Implement data normalization here.
X_train = np.array(X_train / 255.0 - 0.5 )
X_val = np.array(X_val / 255.0 - 0.5 )
X_test = np.array(X_test / 255.0 - 0.5 )

In [113]:
# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(math.isclose(np.min(X_train), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_train), 0.5, abs_tol=1e-5)), "The range of the training data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))
assert(math.isclose(np.min(X_val), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_val), 0.5, abs_tol=1e-5)), "The range of the validation data is: %.1f to %.1f" % (np.min(X_val), np.max(X_val))

## Build a Two-Layer Feedfoward Network

The code you've written so far is for data processing, not specific to Keras. Here you're going to build Keras-specific code.

Build a two-layer feedforward neural network, with 128 neurons in the fully-connected hidden layer. 

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [114]:
# TODO: Build a two-layer feedforward neural network with Keras here.
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten

model = Sequential()
model.add(Flatten(input_shape=(32, 32, 3)))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(43))
model.add(Activation('softmax'))

In [115]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
dense_layers = []
for l in model.layers:
    if type(l) == Dense:
        dense_layers.append(l)
assert(len(dense_layers) == 2), "There should be 2 Dense layers."
d1 = dense_layers[0]
d2 = dense_layers[1]
assert(d1.input_shape == (None, 3072))
assert(d1.output_shape == (None, 128))
assert(d2.input_shape == (None, 128))
assert(d2.output_shape == (None, 43))

last_layer = model.layers[-1]
assert(last_layer.activation.__name__ == 'softmax'), "Last layer should be softmax activation, is {}.".format(last_layer.activation.__name__)

In [116]:
# Debugging
for l in model.layers[1:]:
    print(l.name, l.input_shape, l.output_shape, l.activation)

dense_31 (None, 3072) (None, 128) <function linear at 0x7f4e2eeab268>
activation_31 (None, 128) (None, 128) <function relu at 0x7f4e2eeab048>
dense_32 (None, 128) (None, 43) <function linear at 0x7f4e2eeab268>
activation_32 (None, 43) (None, 43) <function softmax at 0x7f4e2ee9bd08>


## Train the Network
Compile and train the network for 2 epochs. [Use the `adam` optimizer, with `categorical_crossentropy` loss.](https://keras.io/models/sequential/)

Hint 1: In order to use categorical cross entropy, you will need to [one-hot encode the labels](https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py).

Hint 2: In order to pass the input images to the fully-connected hidden layer, you will need to [reshape the input](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py).

Hint 3: Keras's `.fit()` method returns a `History.history` object, which the tests below use. Save that to a variable named `history`.

In [117]:
# TODO: Compile and train the model here.
import numpy as np
from sklearn.preprocessing import LabelBinarizer
label_binarizer = LabelBinarizer()
y_train_hot = label_binarizer.fit_transform(y_train)
y_val_hot = label_binarizer.fit_transform(y_val)

'''
from keras.utils import np_utils
Y_train = np_utils.to_categorical(y_train, 43)
print(np.shape(Y_train))
'''
model.summary()
model.compile('adam', 'categorical_crossentropy', ['accuracy'])
history = model.fit(X_train, y_train_hot,batch_size=128, epochs=20, validation_data=(X_val,y_val_hot))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_16 (Flatten)         (None, 3072)              0         
_________________________________________________________________
dense_31 (Dense)             (None, 128)               393344    
_________________________________________________________________
activation_31 (Activation)   (None, 128)               0         
_________________________________________________________________
dense_32 (Dense)             (None, 43)                5547      
_________________________________________________________________
activation_32 (Activation)   (None, 43)                0         
Total params: 398,891
Trainable params: 398,891
Non-trainable params: 0
_________________________________________________________________
Train on 27839 samples, validate on 6960 samples
Epoch 1/20
27839/27839 [==============================] - 1s 49us/step - loss: 1.8719 -

In [118]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['acc'][-1] > 0.92), "The training accuracy was: %.3f" % history.history['acc'][-1]
assert(history.history['val_acc'][-1] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: (fill in here)

## Congratulations
You've built a feedforward neural network in Keras!

Don't stop here! Next, you'll add a convolutional layer to drive.py.

## Convolutions
Build a new network, similar to your existing network. Before the hidden layer, add a 3x3 [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters and valid padding.

Then compile and train the network.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

Hint 2: Now that the first layer of the network is a convolutional layer, you no longer need to reshape the input images before passing them to the network. You might need to reload your training data to recover the original shape.

Hint 3: Add a [`Flatten()` layer](https://keras.io/layers/core/#flatten) between the convolutional layer and the fully-connected hidden layer.

In [119]:
# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Dropout

# Build the Fully Connected Neural Network in Keras Here
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(32, 32, 3),
                 border_mode='VALID'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(43))
model.add(Activation('softmax'))
# TODO: Compile and train the model here.
model.summary()
model.compile('adam', 'categorical_crossentropy', ['accuracy'])
history = model.fit(X_train, y_train_hot,batch_size=128, epochs=5, validation_data=(X_val,y_val_hot))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
flatten_17 (Flatten)         (None, 28800)             0         
_________________________________________________________________
dense_33 (Dense)             (None, 128)               3686528   
_________________________________________________________________
activation_33 (Activation)   (None, 128)               0         
_________________________________________________________________
dense_34 (Dense)             (None, 43)                5547      
_________________________________________________________________
activation_34 (Activation)   (None, 43)                0         
Total params: 3,692,971
Trainable params: 3,692,971
Non-trainable params: 0
_________________________________________________________________


/home/jst/share/project/udacity/install/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, kernel_size=(3, 3), padding="VALID", input_shape=(32, 32, 3..., activation="relu")`
  # This is added back by InteractiveShellApp.init_path()


Train on 27839 samples, validate on 6960 samples
Epoch 1/5
27839/27839 [==============================] - 3s 111us/step - loss: 1.2306 - acc: 0.6801 - val_loss: 0.4977 - val_acc: 0.8608
Epoch 2/5
27839/27839 [==============================] - 3s 102us/step - loss: 0.2961 - acc: 0.9296 - val_loss: 0.2693 - val_acc: 0.9306
Epoch 3/5
27839/27839 [==============================] - 3s 103us/step - loss: 0.1637 - acc: 0.9599 - val_loss: 0.1986 - val_acc: 0.9506
Epoch 4/5
27839/27839 [==============================] - 3s 103us/step - loss: 0.1021 - acc: 0.9777 - val_loss: 0.1653 - val_acc: 0.9585
Epoch 5/5
27839/27839 [==============================] - 3s 102us/step - loss: 0.0699 - acc: 0.9850 - val_loss: 0.1349 - val_acc: 0.9697


In [120]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: (fill in here)

## Pooling
Re-construct your network and add a 2x2 [pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

Then compile and train the network.

In [121]:
# TODO: Re-construct the network and add a pooling layer after the convolutional layer.
# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Dropout

# Build the Fully Connected Neural Network in Keras Here
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(32, 32, 3),
                 border_mode='VALID'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(43))
model.add(Activation('softmax'))
# TODO: Compile and train the model here.
model.summary()
model.compile('adam', 'categorical_crossentropy', ['accuracy'])
history = model.fit(X_train, y_train_hot,batch_size=128, epochs=5, validation_data=(X_val,y_val_hot))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
flatten_18 (Flatten)         (None, 7200)              0         
_________________________________________________________________
dense_35 (Dense)             (None, 128)               921728    
_________________________________________________________________
activation_35 (Activation)   (None, 128)               0         
_________________________________________________________________
dense_36 (Dense)             (None, 43)                5547      
_________________________________________________________________
activation_36 (Activation)   (None, 43)                0         
Total para

/home/jst/share/project/udacity/install/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, kernel_size=(3, 3), padding="VALID", input_shape=(32, 32, 3..., activation="relu")`
  if sys.path[0] == '':


Train on 27839 samples, validate on 6960 samples
Epoch 1/5
27839/27839 [==============================] - 2s 84us/step - loss: 1.5867 - acc: 0.5900 - val_loss: 0.7328 - val_acc: 0.7843
Epoch 2/5
27839/27839 [==============================] - 2s 85us/step - loss: 0.4545 - acc: 0.8914 - val_loss: 0.3449 - val_acc: 0.9145
Epoch 3/5
27839/27839 [==============================] - 2s 77us/step - loss: 0.2460 - acc: 0.9440 - val_loss: 0.2377 - val_acc: 0.9414
Epoch 4/5
27839/27839 [==============================] - 2s 81us/step - loss: 0.1611 - acc: 0.9649 - val_loss: 0.1996 - val_acc: 0.9519
Epoch 5/5
27839/27839 [==============================] - 2s 76us/step - loss: 0.1172 - acc: 0.9751 - val_loss: 0.1760 - val_acc: 0.9599


In [122]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: (fill in here)

## Dropout
Re-construct your network and add [dropout](https://keras.io/layers/core/#dropout) after the pooling layer. Set the dropout rate to 50%.

In [123]:
# TODO: Re-construct the network and add a pooling layer after the convolutional layer.
# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Dropout

# Build the Fully Connected Neural Network in Keras Here
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(32, 32, 3),
                 border_mode='VALID'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(43))
model.add(Activation('softmax'))
# TODO: Compile and train the model here.
model.summary()
model.compile('adam', 'categorical_crossentropy', ['accuracy'])
history = model.fit(X_train, y_train_hot,batch_size=128, epochs=5, validation_data=(X_val,y_val_hot))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
flatten_19 (Flatten)         (None, 7200)              0         
_________________________________________________________________
dense_37 (Dense)             (None, 128)               921728    
_________________________________________________________________
activation_37 (Activation)   (None, 128)               0         
_________________________________________________________________
dense_38 (Dense)             (None, 43)                5547      
__________

/home/jst/share/project/udacity/install/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, kernel_size=(3, 3), padding="VALID", input_shape=(32, 32, 3..., activation="relu")`
  if sys.path[0] == '':


Train on 27839 samples, validate on 6960 samples
Epoch 1/5
27839/27839 [==============================] - 3s 96us/step - loss: 1.6761 - acc: 0.5639 - val_loss: 0.7182 - val_acc: 0.8296
Epoch 2/5
27839/27839 [==============================] - 2s 84us/step - loss: 0.5389 - acc: 0.8621 - val_loss: 0.3871 - val_acc: 0.8980
Epoch 3/5
27839/27839 [==============================] - 2s 88us/step - loss: 0.3260 - acc: 0.9166 - val_loss: 0.2564 - val_acc: 0.9506
Epoch 4/5
27839/27839 [==============================] - 3s 91us/step - loss: 0.2319 - acc: 0.9408 - val_loss: 0.1974 - val_acc: 0.9619
Epoch 5/5
27839/27839 [==============================] - 3s 96us/step - loss: 0.1864 - acc: 0.9522 - val_loss: 0.1663 - val_acc: 0.9659


In [124]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: (fill in here)

## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

**Best Validation Accuracy:** (fill in here)

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: The `evaluate()` method should return an array of numbers. Use the `metrics_names()` method to get the labels.

In [125]:
# TODO: Load test data
    
# TODO: Preprocess data & one-hot encode the labels
import numpy as np
from sklearn.preprocessing import LabelBinarizer
label_binarizer = LabelBinarizer()
y_test_hot = label_binarizer.fit_transform(y_test)

# TODO: Evaluate model on test data
score = model.evaluate(X_test, y_test_hot, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


for metric_i in range(len(model.metrics_names)):
    metric_name = model.metrics_names[metric_i]
    metric_value = score[metric_i]
    print('{}: {}'.format(metric_name, metric_value))

12630/12630 [==============================] - 1s 103us/step
Test loss: 0.5149058224469636
Test accuracy: 0.8799683294688909
loss: 0.5149058224469636
acc: 0.8799683294688909


**Test Accuracy:** (fill in here)

## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.